In [1]:
# %load plotFluxes.py
#!/usr/bin/env python

%matplotlib inline
import numpy as np
import numpy.polynomial.polynomial as poly
import math
import matplotlib.pyplot as plt
from plotFlux import readFlux
from plotFlux import getAverageVal
from plotFlux import getData
from plotFlux import getStdDev

#SEP added stuff here...

%matplotlib inline

# Parameters are all obtained from Callen Nuc. Fusion 2010
# This is a DIII-D H-mode before the onset of an ELM

R0=1.7 # Magnetic axis, meters
R=2.2 # location of cold impurities, near separatrix
Bt0= 2. # Teslas, B at R0
mu = 1. # m/mp for bulk ions
Z = 1. # This is Z for the bulk ions
Ti = 1000. # bulk ion temp in eV at cold impurity location
c = 0.001 # c=nI/n0, concentration of impurity
# Flux = Flux_GENE x n0 vt (rhoi/R)^2
# R=L_ref, n0=n_ref, vt=vt_ref, rhoi_ref
# vt=979.0/sqrt(mu) x sqrt(Ti)
# I use mu=2 in this formula because tokamaks typically run with D
# rhoi=1.02 x sqrt(mu Ti)/(Z B)
# factor= Flux/(nI*Flux_GENE) = 1/c vt (rhoi/R)^2

# a little tricky... We want v_pinch.  Flux = v_pinch x nI
# Flux_GENE = v_pinch x n0.  v_pinch = Flux_GENE/(nI/n0) x vt (rhoi/R)^2

B = Bt0*R0/R*10000. # Plasma formulary formulas use Gauss
rhoi = 1.02*np.sqrt(mu*Ti)/(Z*B)
vt=9790.*np.sqrt(Ti/mu)

factor = vt/c*(rhoi/R)**2.

print("rhoi=",rhoi)
print("vt=",vt)
print("B=",B)
print("factor=",factor)

#End of SEP added stuff

dataType  = 0 #Particle flux.
avgFlux   = []
stdDev    = []
ions      = ["H$^+$", "Custom$^{+6}$", "Be$^{+4}$", "Ne$^{+10}$", "Ar$^{+15}$", "Mo$^{+31}$", "W$^{+40}$"]
kineticDataFiles = ["nrgsummary_proton.dat",     "nrgsummary_custom.dat",
             "nrgsummary_beryllium.dat",  "nrgsummary_neon.dat",
             "nrgsummary_argon.dat",      "nrgsummary_molybdenum.dat",
             "nrgsummary_tungsten.dat"]
qmRat     = [1/1, 6/8, 4/9.0121820, 10/20.17970, 15/39.948, 31/95.95, 40/183.84]
startT    = [.53, .64, .37, .16, .16, .13, .16] #% location to start at. For ignoring linear phase.
ltRat     = [6.96, 6.96, 6.96, 6.96, 6.96, 6.96, 6.96] #Ratio of R/L_T = omt to normalize flux different from GENE.
densRat   = [.001, .001, .001, .001, .001, .001, .001] #Ratio of n_imp/n_e to normalize flux to impurities unlike e- from GENE.

#Add some adiabatic info too.
adiabaticIons = ["H$^+$", "Ne$^{+10}$", "W$^{+40}$"]
adiabaticDataFiles = ["nrgsummary_proton_ad.dat", "nrgsummary_neon_ad.dat",
                      "nrgsummary_tungsten_ad.dat"]
qmRatAd   = [1/1, 10/20.17970, 40/183.84]
startTAd  = [.114, .25, .118]
ltRatAd   = [6.96, 6.96, 6.96]
densRatAd = [.001, .001, .001]
avgFluxAd = []
stdDevAd  = []

for i,ion in enumerate(ions):
   data = []
   readFlux(kineticDataFiles[i], data) 
   [t, fluxData] = getData(dataType, data)
   startIndex = math.ceil(startT[i]*len(fluxData)) #Get index closest to startT fraction of total time.
   fluxData = fluxData[startIndex:]
   fluxData = np.array(fluxData) #Need to make a numpy array to perform multiplication by a scalar...
   fluxData = fluxData*factor #* ((1/ltRat[i])**2) * (1/densRat[i]) #Normalize flux for L_T and n_imp.
   avgFlux.append(getAverageVal(fluxData))
   stdDev.append(getStdDev(fluxData))

for i, ion in enumerate(adiabaticIons):
   data = []
   readFlux(adiabaticDataFiles[i], data) 
   [t, fluxDataAd] = getData(dataType, data)
   startIndex = math.ceil(startT[i]*len(fluxDataAd)) #Get index closest to startT fraction of total time.
   fluxDataAd = fluxDataAd[startIndex:]
   fluxDataAd = np.array(fluxDataAd) #Need to make a numpy array to perform multiplication by a scalar...
   fluxDataAd = fluxDataAd*factor #* ((1/ltRatAd[i])**2) * (1/densRatAd[i]) #Normalize flux for L_T and n_imp.
   avgFluxAd.append(getAverageVal(fluxDataAd))
   stdDevAd.append(getStdDev(fluxDataAd))

#Plot kinetic ion points and best fit line.
fig, ax = plt.subplots()
plt.scatter(qmRat, avgFlux)
plt.errorbar(qmRat, avgFlux, yerr=stdDev, linestyle="None")

for i, txt in enumerate(ions):
    ax.annotate(txt, (qmRat[i], avgFlux[i] - stdDev[i])) #Offset y for readibility.

#Generate line of best fit.
qmRatNew = np.linspace(qmRat[len(qmRat)-1]/1.5, 1, 100) #Note: Extend low end a little past last point.
coeffs = poly.polyfit(qmRat, avgFlux, 1)
ffit   = poly.polyval(qmRatNew, coeffs)
plt.plot(qmRatNew, ffit, color='r', label='KineticEl')

#Plot adiabatic ions and line of best fit.
plt.scatter(qmRatAd, avgFluxAd, color='red')
plt.errorbar(qmRatAd, avgFluxAd, yerr=stdDevAd, linestyle="None", color='red')

for i, txt in enumerate(adiabaticIons):
    ax.annotate(txt, (qmRatAd[i], avgFluxAd[i] - stdDevAd[i])) #Offset y for readibility.

#Generate line of best fit.
qmRatAdNew = np.linspace(qmRatAd[len(qmRatAd)-1]/1.5, 1, 100) #Note: Extend low end a little past last point.
coeffsAd = poly.polyfit(qmRatAd, avgFluxAd, 1)
ffit   = poly.polyval(qmRatAdNew, coeffsAd)
plt.plot(qmRatAdNew, ffit, color='blue', label='AdiabaticEl')

plt.gca().invert_xaxis()
plt.gca().invert_yaxis()
textSize = 12
plt.xlabel("[q/m] / [q/m]$_{H^+}$", fontsize=textSize)
plt.ylabel("$v_{pinch}$ (m/s)",    fontsize=textSize)
plt.tight_layout()
plt.legend(loc='upper left')
#plt.savefig('./GoerlerImpurities/FluxPlotAdKin.pdf')
#plt.show()


ModuleNotFoundError: No module named 'plotFlux'